# Nearest Neighbor Search

## Define Nearest Neighbor Search

Given a set of points ${x_1, ..., x_n} \in \mathbb{R}^d$, preprocess them into a data structure $X$ of size `polynomial(n, d)` in time `polynomial(n, d)` such that nearest neighbor queries can be performed in logarithmic time. In other words, given a search query point `q`, radius `r`, and $X$, one can return all $x_i$ such that

$$
\left \|
{q - x_i} \leq r
\right \|
$$

## Find Nearest Neighbor in 2D

Let's assume that we have a random set of points `N = 100`. If we represent these points in a 2D feature space with $x_1 \in [-1, 1]$ and $x_2 \in [-1, 1]$. We want to find all the nearest neighbor within a circle of `radius=1`.  For the purpose of demsontration, I will perform a linear search to find all the exact matches.

In [2]:
import numpy as np
np.random.seed(42)
N = 100

def search_nn_by_dimension(num_points, dim):
    points = np.random.randn(num_points, dim)
    radius = 1
    nearest = []
    for p in points:
        if np.sqrt(p.dot(p)) <= 1:
            nearest.append(p)
    return len(nearest)

print(f"""
number of nearest neighbor within Euclidean distance 1.0: {search_nn_by_dimension(N, 2)}""")


number of nearest neighbor within Euclidean distance 1.0: 41


What if I increase the dimension?

In [3]:
for d in range(3, 10):
    print(f"D={d} number of nearest neighbor within " +
          f"Euclidean distance 1.0: {search_nn_by_dimension(N, d)}"
    )

D=3 number of nearest neighbor within Euclidean distance 1.0: 21
D=4 number of nearest neighbor within Euclidean distance 1.0: 12
D=5 number of nearest neighbor within Euclidean distance 1.0: 5
D=6 number of nearest neighbor within Euclidean distance 1.0: 2
D=7 number of nearest neighbor within Euclidean distance 1.0: 0
D=8 number of nearest neighbor within Euclidean distance 1.0: 0
D=9 number of nearest neighbor within Euclidean distance 1.0: 0


As we increase the dimensionality of feature space, the data points are getting further apart. Nearest neighbor search within a radius `r` will start to reduce its effectiveness. In order to more accurately find nearest neighbor, the search radius must increase but it will also result in higher time complexity for the exact search.

> The curse of dimensionality complicates nearest neighbor search in high dimensional space. It is not possible to quickly reject candidates by using the difference in one coordinate as a lower bound for a distance based on all the dimensions.

## But Why?

Consider feature space as a hyper-cube, the volume of a hyper-cube is simply

$$
V_{\text{hypercube}} = s^d
$$

where $d$ is dimension and $l$ is the length of the side. What about a hypercube? It involves a gamma function $\Gamma$.

$$
V_{\text{hypersphere}} = \frac{\pi^{d/2} r^d}{\Gamma(\frac{d}{2} + 1)}
$$

Using the previous example, let's assume we want to search within the radius of $\frac{s}{2}$. The points are randomly scattered in the feature space, so the probability of finding a point inside the search radius is the ratio of hypersphere volume to hypercube volume.

$$
\frac{V_{\text{hypersphere}}}{V_{\text{hypercube}}} = \frac{\pi^{d/2}}{2^d\Gamma(\frac{d}{2} + 1)}
$$

What does this mean? The probability of finding points inside the search radius decreases as the dimension increases! THe rate decreases exponentially too.

> There are no known exact methods for finding nearest neighbors efficiently. As both the number of points increases and the number of dimensions increase, we fall victim to the curse of dimensionality. In high dimensions, all points are almost equally distant from each other. A good enough solution for many applications is to trade accuracy for efficiency. In approximately nearest neighbors (ANN), we build index structures that narrow down the search space. The implicit neighborhoods in such indexes also help reduce the problem of high dimensions. 

## Approximate Nearest Neighbor

Resources

- [ANNOY ANN Part 1](https://erikbern.com/2015/09/24/nearest-neighbor-methods-vector-models-part-1.html)
- [ANNOY ANN Part 2](https://erikbern.com/2015/10/01/nearest-neighbors-and-vector-models-part-2-how-to-search-in-high-dimensional-spaces.html)


If we care more about search speed, we need to give up accuracy for it. Let's look at a couple of approximate nearest neighbor search. Let's assume we have a set of trained embeddings for movies based on the MovieLens dataset.

In [15]:
import pickle

with open('movies.pickle', 'rb') as f:
    embeddings = pickle.load(f)

print(embeddings['name'].shape)
print(embeddings['vector'].shape)

(1682,)
(1682, 64)


## Linear Search

We can build a linear search with `faiss`.

In [42]:
import faiss

class BruteForceIndex():
    def __init__(self, embeddings, labels):
        self.dimension = embeddings.shape[1]
        self.embeddings = embeddings.astype('float32')
        self.labels = labels
    
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension)
        self.index.add(self.embeddings)
    
    def query(self, q, k=10):
        return self.index.search(q, k)
        # return distances, indices
        
index = BruteForceIndex(embeddings['vector'], embeddings['name'])
index.build()

In [59]:
i = 0
print(f"Query: {embeddings['name'][i]} as query\n")

distances, indices = index.query(embeddings['vector'][i:i+1, :])
for j in range(len(indices)):
    candidates = embeddings['name'][indices[j]]
    dists = distances[j]
    for k in range(len(candidates)):
        print(f"- {candidates[k]} with distance {dists[k]}")

Query: Toy Story (1995) as query

- Toy Story (1995) with distance 0.0
- Rock, The (1996) with distance 1.4708294868469238
- Return of the Jedi (1983) with distance 1.6384427547454834
- Willy Wonka and the Chocolate Factory (1971) with distance 1.7211445569992065
- Phenomenon (1996) with distance 1.7770946025848389
- Star Trek: First Contact (1996) with distance 1.8738785982131958
- Star Wars (1977) with distance 1.8834713697433472
- Hunchback of Notre Dame, The (1996) with distance 1.883762001991272
- Birdcage, The (1996) with distance 1.9082722663879395
- Mars Attacks! (1996) with distance 1.9544236660003662
